# Notebook for making example illustrations of the "OmikronDeltaFullModel" model

In [40]:
%matplotlib widget

import sys
sys.path.append("./../scripts")
import PandemiXFunctions as pf
import PandemiXModelFunctions as pmf

import os


import numpy as np

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
             
             
from scipy.integrate import cumtrapz
# int_I = cumtrapz(np.exp(u_H*ts)*Is,ts, initial=0)
# Hs = np.exp(-u_H*ts) * (r_chr*pars_full['gamma']*int_I + k) 

In [41]:
path_figs = os.getcwd() + '\\Figures\\'
path_figs = path_figs + 'TwoStrainModel_'

In [42]:
# Reload PandemiXModelFunctions, in case changes have been made
import importlib 
importlib.reload(pmf)

<module 'PandemiXModelFunctions' from 'c:\\Users\\rakrpe\\OneDrive - Roskilde Universitet\\Documents\\PandemiX\\GithubRepos\\PandemiX\\modelling\\PandemiXModelFunctions.py'>

# First, each strain by itself, with and without vaccinations

In [43]:
# Initialize everything
V_0 = 0
IS_0 = 0
IV_0 = 0
Y_0 = 0
R01_0 = 0
R10_0 = 0
I01_0 = 0
Y10_0 = 0
S_0 = 1 - V_0 - IS_0 - IV_0 - Y_0 - R01_0 - R10_0 - I01_0 - Y10_0 

init = {    
    'S' : S_0,
    'V' : V_0,
    'IS' : IS_0,
    'IV' : IV_0,
    'Y' : Y_0,
    'R01' : R01_0,
    'R10' : R10_0,
    'I01' : I01_0,
    'Y10' : Y10_0,
}

betas = 2/7
gammas = 1/7
pars = {
    'beta_IS_S' : betas,
    'beta_IV_S' : betas,
    'beta_I01_S' : betas,
    'beta_IS_V' : betas,
    'beta_IV_V' : betas,
    'beta_I01_V' : betas,
    'beta_IS_R01' : betas,
    'beta_IV_R01' : betas,
    'beta_I01_R01' : betas,
    'beta_Y_S' : betas,
    'beta_Y10_S' : betas,
    'beta_Y_R10' : betas,
    'beta_Y10_R10' : betas,
    'gamma_IS' : gammas,
    'gamma_IV' : gammas,
    'gamma_Y' : gammas,
    'gamma_I01' : gammas,
    'gamma_Y10' : gammas,
    }


In [44]:
iniInf = 0.0001

init_onlyI = init.copy()
init_onlyI['S'] -= iniInf 
init_onlyI['IS'] += iniInf 

init_onlyY = init.copy()
init_onlyY['S'] -= iniInf 
init_onlyY['Y'] += iniInf 

iniVacc = 0.3
init_onlyI_vacc = init_onlyI.copy()
init_onlyI_vacc['S'] -= iniVacc
init_onlyI_vacc['V'] += iniVacc

init_onlyY_vacc = init_onlyY.copy()
init_onlyY_vacc['S'] -= iniVacc
init_onlyY_vacc['V'] += iniVacc


In [45]:
tEnd = 200
sch_onlyI = pmf.Scheme('OmikronDeltaFull',init_onlyI,pars,0,tEnd,Changes=[])
sch_onlyY = pmf.Scheme('OmikronDeltaFull',init_onlyY,pars,0,tEnd,Changes=[])
sch_onlyI_vacc = pmf.Scheme('OmikronDeltaFull',init_onlyI_vacc,pars,0,tEnd,Changes=[])
sch_onlyY_vacc = pmf.Scheme('OmikronDeltaFull',init_onlyY_vacc,pars,0,tEnd,Changes=[])

sch_onlyI.simulate()
sch_onlyY.simulate()
sch_onlyI_vacc.simulate()
sch_onlyY_vacc.simulate()

r_oI = sch_onlyI.result 
r_oY = sch_onlyY.result 
r_oIv = sch_onlyI_vacc.result 
r_oYv = sch_onlyY_vacc.result 

In [46]:
fig,ax1 = plt.subplots()
curLabels = [r'$I_S$: Infectious',r'$R_{1,0}$: Recovered',r'$S$: Susceptible']
curColors = ['xkcd:orange','xkcd:green','xkcd:sky blue']
ax1.stackplot(r_oI.t,r_oI.IS,r_oI.R10,r_oI.S,labels=curLabels,colors=curColors)
# ax1.legend(bbox_to_anchor=(1,0.5),loc='center left') 
ax1.legend(bbox_to_anchor=(1,0.5,0.25,0.1),loc='center right',mode='expand') 
ax1.set_ylim([0,1])
ax1.set_xlim([r_oI.t[0],r_oI.t[-1]])
ax1.set_xlabel('t')
ax1.set_ylabel('Share of population')
fig.tight_layout()

fig.savefig(path_figs+'OnlyI')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
fig,ax1 = plt.subplots()
RTs = 1 - r_oIv.IS-r_oIv.IV-r_oIv.R10-r_oIv.S-r_oIv.V
# curLabels = [r'$I_S$: Infectious',r'$R_{1,0}$: Recovered',r'$R_{1,1}$: Immune to both',r'$S$: Susceptible',r'$V$: Vaccinated']
# curColors = ['xkcd:orange','xkcd:green','xkcd:dark green','xkcd:sky blue','xkcd:grey']
# ax1.stackplot(r_oIv.t,r_oIv.IS,r_oIv.R10,RTs,r_oIv.S,r_oIv.V,labels=curLabels,colors=curColors)
# curLabels = [r'$I_S$: Infectious',r'$R_{1,0}$: Recovered',r'$S$: Susceptible',r'$R_{1,1}$: Immune to both',r'$V$: Vaccinated']
# curColors = ['xkcd:orange','xkcd:green','xkcd:sky blue','xkcd:dark green','xkcd:grey']
# ax1.stackplot(r_oIv.t,r_oIv.IS,r_oIv.R10,r_oIv.S,RTs,r_oIv.V,labels=curLabels,colors=curColors)
                
curLabels = [   
                r'$I_S$: Infectious'+'\n(unvaccinated)',
                r'$I_V$: Infectious'+'\n(vaccinated)',
                r'$R_{1,0}$: Recovered',
                r'$R_{1,1}$: Immune to both',
                r'$S$: Susceptible',
                r'$V$: Vaccinated'
                ]
curColors = ['xkcd:orange','xkcd:light orange','xkcd:green','xkcd:dark green','xkcd:sky blue','xkcd:grey']
ax1.stackplot(r_oIv.t,
                r_oIv.IS,
                r_oIv.IV,
                r_oIv.R10,
                RTs,
                r_oIv.S,
                r_oIv.V,
                labels=curLabels,colors=curColors)

# ax1.legend(bbox_to_anchor=(1,0.5),loc='center left') 
ax1.legend(bbox_to_anchor=(1,0.5,0.25,0.1),loc='center right',mode='expand') 
ax1.set_ylim([0,1])
ax1.set_xlim([r_oIv.t[0],r_oIv.t[-1]])
ax1.set_xlabel('t')
ax1.set_ylabel('Share of population')
fig.tight_layout()

fig.savefig(path_figs+'OnlyI_Vacc')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
fig,ax1 = plt.subplots()
curLabels = [r'$Y$: Infectious',r'$R_{0,1}$: Recovered',r'$S$: Susceptible']
curColors = ['xkcd:red','xkcd:light green','xkcd:sky blue']
ax1.stackplot(r_oY.t,r_oY.Y,r_oY.R01,r_oY.S,labels=curLabels,colors=curColors)
# ax1.legend(bbox_to_anchor=(1,0.5),loc='center left') 
ax1.legend(bbox_to_anchor=(1,0.5,0.25,0.1),loc='center right',mode='expand') 
ax1.set_ylim([0,1])
ax1.set_xlim([r_oY.t[0],r_oY.t[-1]])
ax1.set_xlabel('t')
ax1.set_ylabel('Share of population')
fig.tight_layout()

fig.savefig(path_figs+'OnlyY')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
fig,ax1 = plt.subplots()
curLabels = [r'$Y$: Infectious',r'$R_{0,1}$: Recovered',r'$S$: Susceptible',r'$V$: Vaccinated']
curColors = ['xkcd:red','xkcd:light green','xkcd:sky blue','xkcd:gray']
ax1.stackplot(r_oYv.t,r_oYv.Y,r_oYv.R01,r_oYv.S,r_oYv.V,labels=curLabels,colors=curColors)
# ax1.legend(bbox_to_anchor=(1,0.5),loc='center left') 
ax1.legend(bbox_to_anchor=(1,0.5,0.25,0.1),loc='center right',mode='expand') 
ax1.set_ylim([0,1])
ax1.set_xlim([r_oYv.t[0],r_oYv.t[-1]])
ax1.set_xlabel('t')
ax1.set_ylabel('Share of population')
fig.tight_layout()

fig.savefig(path_figs+'OnlyY_vacc')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Subsequent epidemics

In [50]:
tEnd = 350
sch_IthenY = pmf.Scheme('OmikronDeltaFull',init_onlyI,pars,0,tEnd,Changes=[])
sch_YthenI = pmf.Scheme('OmikronDeltaFull',init_onlyY,pars,0,tEnd,Changes=[])

tChange = 150
toAdd = iniInf
dictToAdd =  {  
    'S' : -toAdd,
    'V' : 0,
    'IS' : 0,
    'IV' : 0,
    'Y' : toAdd,
    'R01' : 0,
    'R10' : 0,
    'I01' : 0,
    'Y10' : 0,
}
addY = pmf.Change(tChange, AddVariables=dictToAdd)
dictToAdd =  {  
    'S' : -toAdd,
    'V' : 0,
    'IS' : toAdd,
    'IV' : 0,
    'Y' : 0,
    'R01' : 0,
    'R10' : 0,
    'I01' : 0,
    'Y10' : 0,
}
addI = pmf.Change(tChange, AddVariables=dictToAdd)

sch_IthenY.addChange(addY)
sch_IthenY.simulate()
sch_YthenI.addChange(addI)
sch_YthenI.simulate()
# sch_onlyY = pmf.Scheme('OmikronDeltaFull',init_onlyY,pars,0,tEnd,Changes=[])

In [51]:
curRes = sch_IthenY.result 
RTs = 1 - curRes.y.sum(axis=0)

fig,ax1 = plt.subplots()
curLabels = [   
                r'$I_S$: Infectious'+'\n(unvaccinated)',
                r'$I_V$: Infectious'+'\n(vaccinated)',
                r'$I_{0,1}$: Infectious'+'\n(previous Y)',
                r'$Y$: Infectious',
                r'$Y_{1,0}$: Infectious'+'\n(previous I)',
                r'$R_{1,0}$: Recovered, I',
                r'$R_{0,1}$: Recovered, Y',
                r'$R_{1,1}$: Immune to both',
                r'$S$: Susceptible',
                r'$V$: Vaccinated'
                ]
curColors = ['xkcd:orange','xkcd:light orange','xkcd:yellow','xkcd:red','xkcd:light red','xkcd:green','xkcd:light green','xkcd:dark green','xkcd:sky blue','xkcd:grey']
ax1.stackplot(curRes.t,
                curRes.IS,
                curRes.IV,
                curRes.I01,
                curRes.Y,
                curRes.Y10,
                curRes.R10,
                curRes.R01,
                RTs,
                curRes.S,
                curRes.V,
                labels=curLabels,colors=curColors)

ax1.legend(bbox_to_anchor=(1,0.5,0.25,0.1),loc='center right',mode='expand') 
ax1.set_ylim([0,1])
ax1.set_xlim([curRes.t[0],curRes.t[-1]])
ax1.set_xlabel('t')
ax1.set_ylabel('Share of population')
fig.tight_layout()

fig.savefig(path_figs+'IthenY')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
curRes = sch_YthenI.result 
RTs = 1 - curRes.y.sum(axis=0)

fig,ax1 = plt.subplots()
curLabels = [   
                r'$I_S$: Infectious'+'\n(unvaccinated)',
                r'$I_V$: Infectious'+'\n(vaccinated)',
                r'$I_{0,1}$: Infectious'+'\n(previous Y)',
                r'$Y$: Infectious',
                r'$Y_{1,0}$: Infectious'+'\n(previous I)',
                r'$R_{1,0}$: Recovered, I',
                r'$R_{0,1}$: Recovered, Y',
                r'$R_{1,1}$: Immune to both',
                r'$S$: Susceptible',
                r'$V$: Vaccinated'
                ]
curColors = ['xkcd:orange','xkcd:light orange','xkcd:yellow','xkcd:red','xkcd:light red','xkcd:green','xkcd:light green','xkcd:dark green','xkcd:sky blue','xkcd:grey']
ax1.stackplot(curRes.t,
                curRes.IS,
                curRes.IV,
                curRes.I01,
                curRes.Y,
                curRes.Y10,
                curRes.R10,
                curRes.R01,
                RTs,
                curRes.S,
                curRes.V,
                labels=curLabels,colors=curColors)

ax1.legend(bbox_to_anchor=(1,0.5,0.25,0.1),loc='center right',mode='expand') 
ax1.set_ylim([0,1])
ax1.set_xlim([curRes.t[0],curRes.t[-1]])
ax1.set_xlabel('t')
ax1.set_ylabel('Share of population')
fig.tight_layout()

fig.savefig(path_figs+'YthenI')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [53]:
sch_IthenY_vacc = pmf.Scheme('OmikronDeltaFull',init_onlyI_vacc,pars,0,tEnd,Changes=[])
sch_YthenI_vacc = pmf.Scheme('OmikronDeltaFull',init_onlyY_vacc,pars,0,tEnd,Changes=[])

sch_IthenY_vacc.addChange(addY)
sch_IthenY_vacc.simulate()
sch_YthenI_vacc.addChange(addI)
sch_YthenI_vacc.simulate()

In [54]:
curRes = sch_IthenY_vacc.result 
RTs = 1 - curRes.y.sum(axis=0)

fig,ax1 = plt.subplots()
curLabels = [   
                r'$I_S$: Infectious'+'\n(unvaccinated)',
                r'$I_V$: Infectious'+'\n(vaccinated)',
                r'$I_{0,1}$: Infectious'+'\n(previous Y)',
                r'$Y$: Infectious',
                r'$Y_{1,0}$: Infectious'+'\n(previous I)',
                r'$R_{1,0}$: Recovered, I',
                r'$R_{0,1}$: Recovered, Y',
                r'$R_{1,1}$: Immune to both',
                r'$S$: Susceptible',
                r'$V$: Vaccinated'
                ]
curColors = ['xkcd:orange','xkcd:light orange','xkcd:yellow','xkcd:red','xkcd:light red','xkcd:green','xkcd:light green','xkcd:dark green','xkcd:sky blue','xkcd:grey']
ax1.stackplot(curRes.t,
                curRes.IS,
                curRes.IV,
                curRes.I01,
                curRes.Y,
                curRes.Y10,
                curRes.R10,
                curRes.R01,
                RTs,
                curRes.S,
                curRes.V,
                labels=curLabels,colors=curColors)

ax1.legend(bbox_to_anchor=(1,0.5,0.25,0.1),loc='center right',mode='expand') 
ax1.set_ylim([0,1])
ax1.set_xlim([curRes.t[0],curRes.t[-1]])
ax1.set_xlabel('t')
ax1.set_ylabel('Share of population')
fig.tight_layout()

fig.savefig(path_figs+'IthenY_Vacc')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
curRes = sch_YthenI_vacc.result 
RTs = 1 - curRes.y.sum(axis=0)

fig,ax1 = plt.subplots()
curLabels = [   
                r'$I_S$: Infectious'+'\n(unvaccinated)',
                r'$I_V$: Infectious'+'\n(vaccinated)',
                r'$I_{0,1}$: Infectious'+'\n(previous Y)',
                r'$Y$: Infectious',
                r'$Y_{1,0}$: Infectious'+'\n(previous I)',
                r'$R_{1,0}$: Recovered, I',
                r'$R_{0,1}$: Recovered, Y',
                r'$R_{1,1}$: Immune to both',
                r'$S$: Susceptible',
                r'$V$: Vaccinated'
                ]
curColors = ['xkcd:orange','xkcd:light orange','xkcd:yellow','xkcd:red','xkcd:light red','xkcd:green','xkcd:light green','xkcd:dark green','xkcd:sky blue','xkcd:grey']
ax1.stackplot(curRes.t,
                curRes.IS,
                curRes.IV,
                curRes.I01,
                curRes.Y,
                curRes.Y10,
                curRes.R10,
                curRes.R01,
                RTs,
                curRes.S,
                curRes.V,
                labels=curLabels,colors=curColors)

ax1.legend(bbox_to_anchor=(1,0.5,0.25,0.1),loc='center right',mode='expand') 
ax1.set_ylim([0,1])
ax1.set_xlim([curRes.t[0],curRes.t[-1]])
ax1.set_xlabel('t')
ax1.set_ylabel('Share of population')
fig.tight_layout()

fig.savefig(path_figs+'YthenI_Vacc')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# From here, I'll move to another document to simulate various relevant scenarios

# Old stuff below

In [56]:
# ts = sch_onlyI.result.t

# Ss = curScheme.result.y[0,:]
# Vs = curScheme.result.y[1,:]
# ISs = curScheme.result.y[2,:]
# IVs = curScheme.result.y[3,:]
# Ys = curScheme.result.y[4,:]
# R01s = curScheme.result.y[5,:]
# R10s = curScheme.result.y[6,:]
# I01s = curScheme.result.y[7,:]
# Y10s = curScheme.result.y[8,:]
# RTs = 1-curScheme.result.y.sum(axis=0)

In [57]:
# fig,ax1 = plt.subplots()
# # ax1.stackplot(ts,Ss,ISs,R10s)
# # ax1.stackplot(ts,ISs,R10s,Ss,labels=[r'$I_S$: Infectious',r'$R_{1,0}$: Recovered',r'$S$: Susceptible'],colors=['xkcd:orange','xkcd:green','xkcd:sky blue'])
# # ax1.stackplot(ts,ISs,Ys,I01s,Y10s,R10s,R01s,RTs,Ss,colors=['r','r','r','r','g','g','g','b'])
# # ax1.stackplot(ts,ISs,Ys,I01s,Y10s,R10s,R01s,RTs,Ss)
# ax1.stackplot(r_oI.t,r_oI.IS,r_oI.R10,r_oI.S,labels=[r'$I_S$: Infectious',r'$R_{1,0}$: Recovered',r'$S$: Susceptible'],colors=['xkcd:orange','xkcd:green','xkcd:sky blue'])
# ax1.legend(bbox_to_anchor=(1,0.5),loc='center left') 
# ax1.set_ylim([0,1])
# ax1.set_xlim([ts[0],ts[-1]])
# ax1.set_xlabel('t')
# ax1.set_ylabel('Share of population')
# fig.tight_layout()